In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col,to_timestamp
from pyspark.sql.functions import broadcast
spark = SparkSession.builder.appName("Jupyter").getOrCreate()
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

24/12/15 22:42:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


1 - Question

In [2]:
df_matches = spark.read.option("header", "true").csv("/home/iceberg/data/matches.csv")
df_matches.show()
df_matches.printSchema()

+--------------------+--------------------+------------+--------------------+--------------------+-------------+--------------------+--------------+---------+--------------------+
|            match_id|               mapid|is_team_game|         playlist_id|     game_variant_id|is_match_over|     completion_date|match_duration|game_mode|      map_variant_id|
+--------------------+--------------------+------------+--------------------+--------------------+-------------+--------------------+--------------+---------+--------------------+
|11de1a94-8d07-416...|c7edbf0f-f206-11e...|        true|f72e0ef0-7c4a-430...|1e473914-46e4-408...|         true|2016-02-22 00:00:...|          NULL|     NULL|                NULL|
|d3643e71-3e51-43e...|cb914b9e-f206-11e...|       false|d0766624-dbd7-453...|257a305e-4dd3-41f...|         true|2016-02-14 00:00:...|          NULL|     NULL|                NULL|
|d78d2aae-36e4-48a...|c7edbf0f-f206-11e...|        true|f72e0ef0-7c4a-430...|1e473914-46e4-408...|  

In [4]:
df_maps = spark.read.option("header", "true").csv("/home/iceberg/data/maps.csv")
df_maps.show()

+--------------------+-------------------+--------------------+
|               mapid|               name|         description|
+--------------------+-------------------+--------------------+
|c93d708f-f206-11e...|              Urban|Andesia was the c...|
|cb251c51-f206-11e...|     Raid on Apex 7|This unbroken rin...|
|c854e54f-f206-11e...|March on Stormbreak|                NULL|
|c8d69870-f206-11e...| Escape from A.R.C.|Scientists flocke...|
|73ed1fd0-45e5-4bb...|             Osiris|                NULL|
|96c3e3dd-7703-408...|          Blue Team|                NULL|
|1c4f8e19-b046-4f7...|            Glassed|                NULL|
|825065cf-df57-42e...|        Unconfirmed|                NULL|
|9a188f67-1664-4d7...|           Alliance|                NULL|
|2702ea83-2c3e-4fd...|   Before the Storm|                NULL|
|82f8471c-a2ef-408...|            Genesis|                NULL|
|fcd7caa4-37c9-436...|       The Breaking|                NULL|
|7dc80b62-dd39-41d...|          Guardian

In [5]:
result_df = df_matches.join(broadcast(df_maps), "mapid", "inner")
result_df.show()

+--------------------+--------------------+------------+--------------------+--------------------+-------------+--------------------+--------------+---------+--------------------+--------------+--------------------+
|               mapid|            match_id|is_team_game|         playlist_id|     game_variant_id|is_match_over|     completion_date|match_duration|game_mode|      map_variant_id|          name|         description|
+--------------------+--------------------+------------+--------------------+--------------------+-------------+--------------------+--------------+---------+--------------------+--------------+--------------------+
|c7edbf0f-f206-11e...|11de1a94-8d07-416...|        true|f72e0ef0-7c4a-430...|1e473914-46e4-408...|         true|2016-02-22 00:00:...|          NULL|     NULL|                NULL|Breakout Arena|The broadcast of ...|
|cb914b9e-f206-11e...|d3643e71-3e51-43e...|       false|d0766624-dbd7-453...|257a305e-4dd3-41f...|         true|2016-02-14 00:00:...|   

2 - Question (Bucket join match_details, matches, and medal_matches_players on match_id with 16 buckets)

In [6]:
df_match_details=spark.read.option("header", "true").csv("/home/iceberg/data/match_details.csv")
df_matches=spark.read.option("header", "true").csv("/home/iceberg/data/matches.csv")
df_medals_matches_players=spark.read.option("header", "true").csv("/home/iceberg/data/medals_matches_players.csv")

In [ ]:
spark.sql("""DROP TABLE IF EXISTS bootcamp.matches_bucketed""")
MatchesBucketedDDL = """
CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
     match_id STRING,
     is_team_game BOOLEAN,
     playlist_id STRING,
     completion_date TIMESTAMP
 )
 USING iceberg
 PARTITIONED BY (completion_date, bucket(16, match_id));
 """
spark.sql(MatchesBucketedDDL)


df_matches.select(
    col("match_id"), col("is_team_game").cast('boolean'), col("playlist_id"), to_timestamp(col("completion_date")).alias("completion_date"))\
     .write.mode("append")\
     .partitionBy("completion_date")\
    .bucketBy(16, "match_id")\
    .saveAsTable("bootcamp.matches_bucketed")
   
    

